In [1]:
import os, gc, cv2, math, copy, time, random
import pickle
# For data manipulation
import numpy as np, pandas as pd

# Pytorch Imports
import torch, torch.nn as nn, torch.optim as optim
from torch.optim import lr_scheduler

from torch.cuda import amp

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import f1_score,roc_auc_score
# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from utils.utils_2dcnn import *
from utils.model_2dcnn import Net, criterion

def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)

In [2]:
types_ = 'challenge'
# types_ = 'non_challenge'
# Get data dict
if types_=='challenge':
    with open('/ssd8/2023COVID19/Train_Valid_dataset/filter_slice_train_dic1_05_challenge.pickle', 'rb') as f: #train dict challenge
        train_dic = pickle.load(f)
    with open('/ssd8/2023COVID19/Train_Valid_dataset/filter_slice_valid_dic1_05_challenge.pickle', 'rb') as f: #valid dict challenge
        valid_dlc = pickle.load(f)
    train_df = pd.read_csv('/ssd8/2023COVID19/Train_Valid_dataset/filter_slice_train_df_challenge.csv') #train df challenge
    valid_df = pd.read_csv('/ssd8/2023COVID19/Train_Valid_dataset/filter_slice_valid_df_challenge.csv') #valid df challenge
else:
    with open('/ssd8/2023COVID19/Train_Valid_dataset/filter_slice_train_dic1_05.pickle', 'rb') as f: #train dict non-challenge
        train_dic = pickle.load(f)
    with open('/ssd8/2023COVID19/Train_Valid_dataset/filter_slice_valid_dic1_05.pickle', 'rb') as f: #valid dict non-challenge
        valid_dlc = pickle.load(f)
    train_df = pd.read_csv('/ssd8/2023COVID19/Train_Valid_dataset/filter_slice_train_df.csv') #train df non-challenge
    valid_df = pd.read_csv('/ssd8/2023COVID19/Train_Valid_dataset/filter_slice_valid_df.csv') #valid df non-challenge
print(len(train_dic), len(valid_dlc))

3028 704


In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "3,4"
set_seed()
job=51  
CONFIG = {"seed": 2022,
        "img_size": 384, #image size: 512
        "train_batch_size": 8, #16
        "valid_batch_size": 16,
        "n_accumulate": 1, #2
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "train_batch":16,
            }

data_transforms = {
    "valid": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),

        A.Normalize(),
        ToTensorV2()], p=1.)
}


print("="*10, "loading *model*", "="*10)
model=Net()
# model = nn.DataParallel(model, device_ids=[0,1])
model=model.to(CONFIG['device'])
# pred_path='/home/chihyi111/covid_2023/train_code_challenge/model/loss/job_51_effnetb3a_size256_challenge[DataParallel].bin'#weight path
pred_path = "/ssd8/2023COVID19/CT-COVID19-Classification/train_code_fix/model/f1/job_51_effnetb3a_size384_challenge[DataParallel]_146.bin"
# pred_path ='/ssd8/2023COVID19/CT-COVID19-Classification/train_code_fix/model/f1/job_51_effnetb3a_size384_challenge[DataParallel].bin'#weight path
# pred_path='/ssd8/2023COVID19/CT-COVID19-Classification/train_code_fix/model/f1/job_51_effnetb3a_size256_challenge[DataParallel].bin'#weight path
ck_point = torch.load(pred_path)
model.load_state_dict(ck_point)

========== loading *model* ==========


<All keys matched successfully>

In [4]:
@torch.inference_mode()
def pred_one(model, dataloader, device):
    model.eval()
    true_y=[]
    pred_y=[]
    with torch.no_grad():
        for data in tqdm(dataloader):
            ct_b, img_b, c, h, w = data['image'].size()
            data_img = data['image'].reshape(-1, c, h, w)
            data_label = data['label'].reshape(-1,1)
            images = data_img.to(device, dtype=torch.float)
            labels = data_label.to(device, dtype=torch.float)
            outputs = model(images)
            true_y.append(labels.cpu().numpy())
            pred_y.append(torch.sigmoid(outputs).cpu().numpy())
    true_y=np.concatenate(true_y)
    pred_y=np.concatenate(pred_y)
    gc.collect()
    # print(pred_y.shape)
    true_y=np.array(true_y).reshape(-1,1)
    true_y=np.array(true_y).reshape(-1,img_b)
    true_y=true_y.mean(axis=1)
    pred_y=np.array(pred_y).reshape(-1,1)
    # print(pred_y.shape)
    pred_y=np.array(pred_y).reshape(-1,img_b)
    # print(pred_y.shape)
    pred_y=pred_y.mean(axis=1)
    # print(pred_y.shape)
    return true_y,pred_y

total_pred=[]
train_loader, valid_loader = prepare_loaders_eval(CONFIG, train_df, train_dic, valid_df, valid_dlc, data_transforms)
for i in range(3):
    true_y,pred_y=pred_one(model, valid_loader, device=CONFIG['device'])
    total_pred.append(pred_y)
for i in range(len(total_pred)):
    print(f1_score(np.array(true_y),np.round(total_pred[i]),average='macro'))   

tn, fp, fn, tp = confusion_matrix(np.array(true_y), np.round(np.mean(total_pred,axis=0))).ravel()
print("Mean F1-Score: {}".format(f1_score(np.array(true_y),np.round(np.mean(total_pred,axis=0)),average='macro')))
print("Negative Accuracy: {}".format(tn/(tn+fp)))#Specificity
print("Positive Accuracy: {}".format(tp/(tp+fn)))#Sensitivity

100%|██████████| 44/44 [00:55<00:00,  1.26s/it]

0.8645790537347389
0.8627288724021497
0.8672291095211835
Mean F1-Score: 0.863084395871281
Negative Accuracy: 0.9530916844349681
Positive Accuracy: 0.7446808510638298


In [ ]:
# ============[baseline]==============
# model: 2dcnn(efficientnet_b3a)
# LR: (0.0001, decay:0.0005)
# batch size: 8
# ------------------------------------
# non-challenge dataset:
# [256x256] [DataParallel] (weight =  best f1 socre checkpoint)
# Mean F1-Score: 0.9200561009817672
# Negative Accuracy: 0.9774436090225563
# Positive Accuracy: 0.8545454545454545

# [384x384] [DataParallel] (weight =  best f1 socre checkpoint
# Mean F1-Score: 0.9305508039685255
# Negative Accuracy: 0.9721030042918455
# Positive Accuracy: 0.8739130434782608
# ------------------------------------
# challenge dataset:
# [256x256] [DataParallel] (weight =  best f1 socre checkpoint)
# Mean F1-Score: 0.8918847674637544
# Negative Accuracy: 0.9377682403433476
# Positive Accuracy: 0.8391304347826087

# [384x384] [DataParallel] (weight =  best f1 socre checkpoint
# Mean F1-Score: 0.9271816605663649
# Negative Accuracy: 0.943609022556391
# Positive Accuracy: 0.9090909090909091